Vamos a hacer una función que nos regrese la formula de integración de Newton Cotes de orden n

Si recordamos, lo que debemos hacer para obtenerla es primero interpolar la función utilizando n+1 puntos ${x_{0},x_{1},...,x_{n}}$ los cuales tienen un espaciado regular: $x_{i} = x_{i-1}+h$ donde $h = \frac{x_{n}-x_{0}}{n}$ la interpolación es la de Lagrange. Posteriormente hacemos la integral desde $x_{0}$ a $x_{n}$ del polinomio. 

In [36]:
function xSubscriptSym(i)
   a = "x_"
   b = string(i)
    
    return string(a,b)
    
end

xSubscriptSym (generic function with 1 method)

In [29]:
#=Hacemos una rutina que interpole una funcion con n puntos, usando un espaciado regular=#

using SymPy

function lagrangeInterpolation(N)
    g = SymFunction("f")
    x0 = Sym("x_0")
    x = Sym("x")
    points = []
    push!(points,x0)
    h = Sym("h")
    #=We create all the symbols for each point=#
    
    for i in 1:N
       push!(points,Sym(xSubscriptSym(i))) 
    end
    
    sum = SymFunction("sum")
    lagrangeTerm = SymFunction("lagrangeTerm")
    lagrangeTerm = 1
    sum = 0
    
    for i in 1:N+1
        lagrangeTerm  = 1
        for j in 1:N+1
            if(j!=i)
                
               lagrangeTerm = lagrangeTerm*(x-points[j])/((points[i])-points[j])
               lagrangeTerm = subs(lagrangeTerm,points[j]=>points[1]+(j-1)*h)
               lagrangeTerm = subs(lagrangeTerm,points[i]=>points[1]+(i-1)*h) 
            
            end
    end
          sum+=g(points[i])*lagrangeTerm
end
    return sum 
end

lagrangeInterpolation (generic function with 1 method)

In [30]:
lagrangeInterpolation(2)

  (x - x_0)*(-2*h + x - x_0)*f(x_1)   (x - x_0)*(-h + x - x_0)*f(x_2)   (-2*h 
- --------------------------------- + ------------------------------- + ------
                   2                                   2                      
                  h                                 2*h                       

+ x - x_0)*(-h + x - x_0)*f(x_0)
--------------------------------
              2                 
           2*h                  

In [31]:
#=Aqui obtendremos Newton Cotes=#
function newtonCotes(N)
    
    l = lagrangeInterpolation(N)
    x0 = Sym("x_0")
    h = Sym("h")
    xn =x0 +N*h
    
    newtonCotes = simplify(integrate(l,(Sym("x"),x0,xn)))
    
    
    
    return newtonCotes
end


newtonCotes (generic function with 1 method)

In [32]:
#=La regla de simpson=#
newtonCotes(2)

h*(f(x_0) + 4*f(x_1) + f(x_2))
------------------------------
              3               

In [33]:
#=La regla del trapecio=#

newtonCotes(1)

h*(f(x_0) + f(x_1))
-------------------
         2         

In [85]:
#=Ahora queremos hacer lo mismo, pero que f, este definida para poder utilizarla, hacemos entonces las
versiones con funciones definidas de las rutinas anteriores. =#

#=Hacemos una rutina que interpole una funcion con n puntos, usando un espaciado regular=#

using SymPy

function lagrangeInterpolationDefined(N,f)
    x0 = Sym("x_0")
    x = Sym("x")
    g = f(x)
    points = []
    push!(points,x0)
    h = Sym("h")
    #=We create all the symbols for each point=#
    
    for i in 1:N
       push!(points,Sym(xSubscriptSym(i))) 
    end
    
    sum = SymFunction("sum")
    lagrangeTerm = SymFunction("lagrangeTerm")
    lagrangeTerm = 1
    sum = 0
    
    for i in 1:N+1
        lagrangeTerm  = 1
        for j in 1:N+1
            if(j!=i)
                
               lagrangeTerm = lagrangeTerm*(x-points[j])/((points[i])-points[j])
               lagrangeTerm = subs(lagrangeTerm,points[j]=>points[1]+(j-1)*h)
               lagrangeTerm = subs(lagrangeTerm,points[i]=>points[1]+(i-1)*h) 
            
            end
    end
          sum+=g(points[i])*lagrangeTerm
end
    return sum 
end

function newtonCotesDefined(N,f)
    
    l = lagrangeInterpolation(N,f)
    x0 = Sym("x_0")
    h = Sym("h")
    xn =x0 +N*h
    
    newtonCotes = simplify(integrate(l,(Sym("x"),x0,xn)))
    
    
    
    return newtonCotes
end



newtonCotesDefined (generic function with 1 method)

In [109]:
function newtonCotesNumerical(f,x0,xf,order)
    points = linspace(x0,xf,order+1)
    g = newtonCotesDefined(order,f)
    h = (xf-x0)/(order)
    g = subs(g,Sym("h")=>h)
    for i in 1:size(points,1)
       g = subs(g,Sym(xSubscriptSym(i-1))=>points[i])
    end
    return g
end

newtonCotesNumerical (generic function with 2 methods)

In [110]:
#=Probemos con el tipico ejemplo de x^2 de 0 a 2, la respuesta debe ser 8/3  = 2.666=#

#=Orden 1=#
println("Orden 1 ",newtonCotesNumerical(x->x^2,0,2,1))
#=Orden 2=#
println("Orden 2 ",newtonCotesNumerical(x->x^2,0,2,2))
#=Orden 3=#
println("Orden 3 ",newtonCotesNumerical(x->x^2,0,2,3))

#=A partir de orden 2, el valor de la integral es cuadrada, porque la aproximacion cuadratica a la funcion 
x^2 es la misma funcion=#

Orden 1 4.00000000000000
Orden 2 2.66666666666667
Orden 3 2.66666666666667


0.0
2.0


LoadError: [91mMethodError: no method matching lambdify(::Base.#sin)[0m
Closest candidates are:
  lambdify([91m::SymPy.Sym[39m) at /home/juser/.julia/v0.6/SymPy/src/lambdify.jl:143
  lambdify([91m::SymPy.Sym[39m, [91m::Any[39m; typ, fns, values) at /home/juser/.julia/v0.6/SymPy/src/lambdify.jl:143[39m